In [1]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [2]:
!pip install pymysql
!pip install -q sklearn
!pip install tensorflow_addons 
!pip install pyyaml h5py  # HDF5 포맷으로 모델을 저장하기 위해서 필요합니다


     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 1.1 MB 31.2 MB/s 


# 라이브러리 import

In [3]:
!mkdir -p saved_model

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import feature_column
from tensorflow.keras import layers
import tensorflow_addons as tfa


import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
import pymysql


# 데이터 전처리

In [6]:
conn = pymysql.connect(
    user = 'orlab',
    passwd = 'orlabghkdlxld123!',
    host = 'orlab.cl9jlcc8q0pr.ap-northeast-2.rds.amazonaws.com',
    db = 'Dataset_final',
    charset = 'utf8'
)
curs = conn.cursor(pymysql.cursors.DictCursor)
curs1 = conn.cursor(pymysql.cursors.DictCursor)

curs.execute("select * from DS_Project_Company_2016_1")

df = curs.fetchall()
df = pd.DataFrame(df)


In [7]:
df

,index,Project_ID,Year,N_of_SCI,N_of_Paper,N_Patent_App,N_Patent_Reg,N_of_Korean_Patent,N_of_Inter_Patent,N_of_Patent,Multi_Year,RnD_Org,Region,STP_Code_11,STP_Code_1_Weight,STP_Code_21,STP_Code_2_Weight,Application_Area_1,Application_Area_1_Weight,Application_Area_2,Application_Area_2_Weight,Application_Area_3,Application_Area_3_Weight,Green_Tech,SixT_2,Econ_Social,National_Strategy_2,RnD_Stage,Cowork_Cor,Cowork_Uni,Cowork_Inst,Cowork_Abroad,Log_RnD_Fund,Log_Duration,Cowork_etc,Sales,Income,Asset,Capital,Sales_Income_Ratio,Asset_Income_Ratio,Sales_Operation_Ratio,Expense_Ratio,Debt_Ratio,IPO,Comp_Type,Listed_Market,Administration,External_Audit,Survival,Venture,Innobiz,Mainbiz,Employees,Closed,Ten_Industry_1,Ten_Industry_11,Ten_Industry_111,Researcher,Comm_Success,Comm_Success_Code1_4,Comm_Success_Code2_5,Comm_Success_Code3_6
0,0,1275000075,2013.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,5,1,SA01,50.0,LA02,30.0,X10,50.0,X99,50.0,None,0.0,999,20100,8,60000,3,N,N,N,N,18.472324,5.897157,N,5.195160e+05,113112.0,1.273918e+06,517728.0,21.772573,8.879064,22.744439,0.638814,0.593594,일반,주식,대상아님,N,N,Y,N,N,N,4,일반과세자,M,M70,M70111,중소기업,1,0,1,0
1,1,1345213293,2013.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3,5,LB01,80.0,LB03,20.0,Y01,100.0,None,0.0,None,0.0,999,20100,6,60000,1,N,N,N,N,17.353742,7.509884,N,1.117283e+08,7991778.0,9.128148e+07,58595026.0,7.152867,8.755092,5.729728,0.034798,0.358084,비영리단체,기타,대상아님,N,N,Y,N,N,N,52,일반과세자,M,M70,M70111,대학,1,1,0,0
2,2,1345231051,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3,2,SI02,100.0,None,0.0,Y15,100.0,None,0.0,None,0.0,999,10400,7,60000,4,N,N,N,N,19.721267,7.509336,N,8.209287e+07,7862077.0,3.143850e+08,275071785.0,9.577053,2.500780,-10.851424,1.108514,0.125048,비영리단체,학교,대상아님,N,N,Y,N,N,N,562,일반과세자,P,P85,P85302,대학,1,0,1,0
3,3,1345231052,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3,1,SI99,100.0,None,0.0,Y17,100.0,None,0.0,None,0.0,999,60200,7,60000,4,N,N,N,N,19.713930,7.509336,N,1.364778e+06,166841.0,1.041825e+06,922162.0,12.224772,16.014302,1.650671,0.146740,0.114858,비영리단체,학교,대상아님,N,N,Y,N,N,N,5,일반과세자,J,J63,J63991,대학,1,1,0,0
4,4,1345231053,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3,13,LB17,100.0,None,0.0,Y02,100.0,None,0.0,None,0.0,999,20200,7,20300,4,N,N,N,N,19.599336,6.997597,N,4.366534e+07,769070.0,2.078784e+07,11890312.0,1.761282,3.699615,2.326291,0.092816,0.428016,비영리단체,기타,대상아님,N,N,Y,N,N,N,0,일반과세자,S,S94,S94990,대학,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49080,65451,9991006112,2016.0,0.0,0.0,8.0,1.0,6.0,3.0,9.0,2,8,8,ED04,50.0,EB03,30.0,Y08,100.0,None,0.0,None,0.0,999,10100,7,10100,3,N,N,N,N,21.041720,6.970731,Y,3.168320e+08,1148516.0,2.776038e+08,120113845.0,0.362500,0.413725,1.578971,0.132878,0.567319,코스닥상장,주식,코스닥,N,Y,Y,N,N,N,106,일반과세자,C,C26,C26295,중견기업,0,0,0,0
49081,65452,9991006114,2016.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,8,8,LC99,100.0,None,0.0,X02,100.0,None,0.0,None,0.0,999,20200,4,60000,3,N,N,N,N,21.808489,6.747588,Y,1.033114e+09,62902751.0,1.422757e+09,982047195.0,6.088657,4.421186,6.716762,0.228559,0.309758,상장,주식,코스피,N,Y,Y,N,N,N,2012,일반과세자,C,C21,C21210,중견기업,0,0,0,0
49082,65454,9991006122,2016.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,8,1,LC03,100.0,None,0.0,X02,100.0,None,0.0,None,0.0,999,20200,4,60000,3,N,N,N,N,21.920324,6.302621,Y,1.055348e+08,10968552.0,1.509786e+08,118356910.0,10.393303,7.264972,10.206674,0.351096,0.216068,외감,주식,대상아님,N,Y,Y,N,N,N,375,일반과세자,C,C21,C21210,중견기업,0,0,0,0
49083,65455,9991006124,2016.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,5,8,EB02,80.0,EA03,20.0,Y08,60.0,Y09,40.0,None,0.0,999,30200,7,10500,3,Y,N,Y,N,20.482467,6.998511,N,4.882565e+06,745474.0,4.860485e+06,2671343.0,15.268081,15.337441,10.514289,0.288234,0.450396,일반,주식,대상아님,N,N,Y,Y,Y,N,73,일반과세자,C,C26,C26121,중소기업,0,0,0,0


In [8]:
df["Year"] = df["Year"].apply(str)

In [9]:
df.columns

Index(['index', 'Project_ID', 'Year', 'N_of_SCI', 'N_of_Paper', 'N_Patent_App',
       'N_Patent_Reg', 'N_of_Korean_Patent', 'N_of_Inter_Patent',
       'N_of_Patent', 'Multi_Year', 'RnD_Org', 'Region', 'STP_Code_11',
       'STP_Code_1_Weight', 'STP_Code_21', 'STP_Code_2_Weight',
       'Application_Area_1', 'Application_Area_1_Weight', 'Application_Area_2',
       'Application_Area_2_Weight', 'Application_Area_3',
       'Application_Area_3_Weight', 'Green_Tech', 'SixT_2', 'Econ_Social',
       'National_Strategy_2', 'RnD_Stage', 'Cowork_Cor', 'Cowork_Uni',
       'Cowork_Inst', 'Cowork_Abroad', 'Log_RnD_Fund', 'Log_Duration',
       'Cowork_etc', 'Sales', 'Income', 'Asset', 'Capital',
       'Sales_Income_Ratio', 'Asset_Income_Ratio', 'Sales_Operation_Ratio',
       'Expense_Ratio', 'Debt_Ratio', 'IPO', 'Comp_Type', 'Listed_Market',
       'Administration', 'External_Audit', 'Survival', 'Venture', 'Innobiz',
       'Mainbiz', 'Employees', 'Closed', 'Ten_Industry_1', 'Ten_Industry_11

In [10]:
df['Sales'] = (df['Sales'] - df['Sales'].mean())/df['Sales'].std()
df['Income'] = (df['Income'] - df['Income'].mean())/df['Income'].std()
df['Asset'] = (df['Asset'] - df['Asset'].mean())/df['Asset'].std()
df['Capital'] = (df['Capital'] - df['Capital'].mean())/df['Capital'].std()

df["Employees"] = np.log1p(df["Employees"])

# feature_columns 생성

In [11]:

feature_columns = []

for header in ['Log_RnD_Fund','Log_Duration','N_of_SCI','N_of_Paper','N_Patent_App','N_Patent_Reg','N_of_Korean_Patent','N_of_Inter_Patent',
               'N_of_Patent','STP_Code_1_Weight','STP_Code_2_Weight','Application_Area_1_Weight','Application_Area_2_Weight','Application_Area_3_Weight','Sales','Income','Asset','Capital',
               'Sales_Income_Ratio','Asset_Income_Ratio', 'Sales_Operation_Ratio', 'Expense_Ratio','Debt_Ratio','Employees']:
    feature_columns.append(feature_column.numeric_column(header, dtype=tf.dtypes.float64 ))

# 범주형 열(Categorical column)은 특정 문자열을 수치형으로 매핑하여 전달

#IPO
df["IPO"] = df["IPO"].apply(str)
IPO = tf.feature_column.categorical_column_with_vocabulary_list(
      'IPO', df['IPO'].unique() )
IPO_hot = tf.feature_column.indicator_column(IPO)
feature_columns.append(IPO_hot)

#Comp_Type
df["Comp_Type"] = df["Comp_Type"].apply(str)
Comp_Type = tf.feature_column.categorical_column_with_vocabulary_list(
      'Comp_Type', df['Comp_Type'].unique() )
Comp_Type_hot = tf.feature_column.indicator_column(Comp_Type)
feature_columns.append(Comp_Type_hot)

#Listed_Market
df["Listed_Market"] = df["Listed_Market"].apply(str)
Listed_Market = tf.feature_column.categorical_column_with_vocabulary_list(
      'Listed_Market', df['Listed_Market'].unique() )
Listed_Market_hot = tf.feature_column.indicator_column(Listed_Market)
feature_columns.append(Listed_Market_hot)

#Administration
df["Administration"] = df["Administration"].apply(str)
Administration = tf.feature_column.categorical_column_with_vocabulary_list(
      'Administration', df['Administration'].unique() )
Administration_hot = tf.feature_column.indicator_column(Administration)
feature_columns.append(Administration_hot)

#External_Audit
df["External_Audit"] = df["External_Audit"].apply(str)
External_Audit = tf.feature_column.categorical_column_with_vocabulary_list(
      'External_Audit', df['External_Audit'].unique() )
External_Audit_hot = tf.feature_column.indicator_column(External_Audit)
feature_columns.append(External_Audit_hot)

#Survival
df["Survival"] = df["Survival"].apply(str)
Survival = tf.feature_column.categorical_column_with_vocabulary_list(
      'Survival', df['Survival'].unique() )
Survival_hot = tf.feature_column.indicator_column(Survival)
feature_columns.append(Survival_hot)

#Venture
df["Venture"] = df["Venture"].apply(str)
Venture = tf.feature_column.categorical_column_with_vocabulary_list(
      'Venture', df['Venture'].unique() )
Venture_hot = tf.feature_column.indicator_column(Venture)
feature_columns.append(Venture_hot)

#Innobiz
df["Innobiz"] = df["Innobiz"].apply(str)
Innobiz = tf.feature_column.categorical_column_with_vocabulary_list(
      'Innobiz', df['Innobiz'].unique() )
Innobiz_hot = tf.feature_column.indicator_column(Innobiz)
feature_columns.append(Innobiz_hot)

#Mainbiz
df["Mainbiz"] = df["Mainbiz"].apply(str)
Mainbiz = tf.feature_column.categorical_column_with_vocabulary_list(
      'Mainbiz', df['Mainbiz'].unique() )
Mainbiz_hot = tf.feature_column.indicator_column(Mainbiz)
feature_columns.append(Mainbiz_hot)

#Closed
df["Closed"] = df["Closed"].apply(str)
Closed = tf.feature_column.categorical_column_with_vocabulary_list(
      'Closed', df['Closed'].unique() )
Closed_hot = tf.feature_column.indicator_column(Closed)
feature_columns.append(Closed_hot)

#Ten_Industry_11
df["Ten_Industry_11"] = df["Ten_Industry_11"].apply(str)
Ten_Industry_11 = tf.feature_column.categorical_column_with_vocabulary_list(
      'Ten_Industry_11', df['Ten_Industry_11'].unique() )
Ten_Industry_11_hot = tf.feature_column.indicator_column(Ten_Industry_11)
feature_columns.append(Ten_Industry_11_hot)

#Researcher
df["Researcher"] = df["Researcher"].apply(str)
Researcher = tf.feature_column.categorical_column_with_vocabulary_list(
      'Researcher', df['Researcher'].unique() )
Researcher_hot = tf.feature_column.indicator_column(Researcher)
feature_columns.append(Researcher_hot)

#Year
df["Year"] = df["Year"].apply(str)
Year = tf.feature_column.categorical_column_with_vocabulary_list(
      'Year', df['Year'].unique() )
Year_hot = tf.feature_column.indicator_column(Year)
feature_columns.append(Year_hot)


#Multi_Year
df["Multi_Year"] = df["Multi_Year"].apply(str)
Multi_Year = tf.feature_column.categorical_column_with_vocabulary_list(
      'Multi_Year', df['Multi_Year'].unique() )
Multi_Year_one_hot = tf.feature_column.indicator_column(Multi_Year)
feature_columns.append(Multi_Year_one_hot)


#RnD_Org
df["RnD_Org"] = df["RnD_Org"].apply(str)
RnD_Org = tf.feature_column.categorical_column_with_vocabulary_list(
      'RnD_Org', df['RnD_Org'].unique() )
RnD_Org_one_hot = tf.feature_column.indicator_column(RnD_Org)
feature_columns.append(RnD_Org_one_hot)

#Region
df["Region"] = df["Region"].apply(str)

Region = tf.feature_column.categorical_column_with_vocabulary_list(
      'Region', df['Region'].unique() )
Region_one_hot = tf.feature_column.indicator_column(Region)
feature_columns.append(Region_one_hot)

#STP_Code_11
df["STP_Code_11"] = df["STP_Code_11"].apply(str)

STP_Code_11 = tf.feature_column.categorical_column_with_vocabulary_list(
      'STP_Code_11', df['STP_Code_11'].unique() )
STP_Code_11_one_hot = tf.feature_column.indicator_column(STP_Code_11)
feature_columns.append(STP_Code_11_one_hot)


#STP_Code_21
df["STP_Code_21"] = df["STP_Code_21"].apply(str)

STP_Code_21 = tf.feature_column.categorical_column_with_vocabulary_list(
      'STP_Code_21', df['STP_Code_21'].unique() )
STP_Code_21_one_hot = tf.feature_column.indicator_column(STP_Code_21)
feature_columns.append(STP_Code_21_one_hot)



#Application_Area_1
df["Application_Area_1"] = df["Application_Area_1"].apply(str)

Application_Area_1 = tf.feature_column.categorical_column_with_vocabulary_list(
      'Application_Area_1', df['Application_Area_1'].unique() )
Application_Area_1_one_hot = tf.feature_column.indicator_column(Application_Area_1)
feature_columns.append(Application_Area_1_one_hot)



#Application_Area_2
df["Application_Area_2"] = df["Application_Area_2"].apply(str)

Application_Area_2 = tf.feature_column.categorical_column_with_vocabulary_list(
      'Application_Area_2', df['Application_Area_2'].unique() )
Application_Area_2_one_hot = tf.feature_column.indicator_column(Application_Area_2)
feature_columns.append(Application_Area_2_one_hot)

#Application_Area_3
df["Application_Area_3"] = df["Application_Area_3"].apply(str)

Application_Area_3 = tf.feature_column.categorical_column_with_vocabulary_list(
      'Application_Area_3', df['Application_Area_3'].unique() )
Application_Area_3_one_hot = tf.feature_column.indicator_column(Application_Area_3)
feature_columns.append(Application_Area_3_one_hot)



#Green_Tech
df["Green_Tech"] = df["Green_Tech"].apply(str)

Green_Tech = tf.feature_column.categorical_column_with_vocabulary_list(
      'Green_Tech', df['Green_Tech'].unique() )
Green_Tech_one_hot = tf.feature_column.indicator_column(Green_Tech)
feature_columns.append(Green_Tech_one_hot)



#SixT_2
df["SixT_2"] = df["SixT_2"].apply(str)
SixT_2 = tf.feature_column.categorical_column_with_vocabulary_list(
      'SixT_2', df['SixT_2'].unique() )
SixT_2_one_hot = tf.feature_column.indicator_column(SixT_2)
feature_columns.append(SixT_2_one_hot)


#Econ_Social
df["Econ_Social"] = df["Econ_Social"].apply(str)
Econ_Social = tf.feature_column.categorical_column_with_vocabulary_list(
      'Econ_Social', df['Econ_Social'].unique() )
Econ_Social_one_hot = tf.feature_column.indicator_column(Econ_Social)
feature_columns.append(Econ_Social_one_hot)

#National_Strategy_2
df["National_Strategy_2"] = df["National_Strategy_2"].apply(str)
National_Strategy_2 = tf.feature_column.categorical_column_with_vocabulary_list(
      'National_Strategy_2', df['National_Strategy_2'].unique() )
National_Strategy_2_one_hot = tf.feature_column.indicator_column(National_Strategy_2)
feature_columns.append(National_Strategy_2_one_hot)

#RnD_Stage
df["RnD_Stage"] = df["RnD_Stage"].apply(str)
RnD_Stage = tf.feature_column.categorical_column_with_vocabulary_list(
      'RnD_Stage', df['RnD_Stage'].unique() )
RnD_Stage_one_hot = tf.feature_column.indicator_column(RnD_Stage)
feature_columns.append(RnD_Stage_one_hot)

#Cowork_Cor
df["Cowork_Cor"] = df["Cowork_Cor"].apply(str)
Cowork_Cor = tf.feature_column.categorical_column_with_vocabulary_list(
      'Cowork_Cor', df['Cowork_Cor'].unique() )
Cowork_Cor_one_hot = tf.feature_column.indicator_column(Cowork_Cor)
feature_columns.append(Cowork_Cor_one_hot)

#Cowork_Uni
df["Cowork_Uni"] = df["Cowork_Uni"].apply(str)
Cowork_Uni = tf.feature_column.categorical_column_with_vocabulary_list(
      'Cowork_Uni', df['Cowork_Uni'].unique() )
Cowork_Uni_one_hot = tf.feature_column.indicator_column(Cowork_Uni)
feature_columns.append(Cowork_Uni_one_hot)

#Cowork_Inst
df["Cowork_Inst"] = df["Cowork_Inst"].apply(str)
Cowork_Inst = tf.feature_column.categorical_column_with_vocabulary_list(
      'Cowork_Inst', df['Cowork_Inst'].unique() )
Cowork_Inst_one_hot = tf.feature_column.indicator_column(Cowork_Inst)
feature_columns.append(Cowork_Inst_one_hot)

#Cowork_Abroad
df["Cowork_Abroad"] = df["Cowork_Abroad"].apply(str)
Cowork_Abroad = tf.feature_column.categorical_column_with_vocabulary_list(
      'Cowork_Abroad', df['Cowork_Abroad'].unique() )
Cowork_Abroad_one_hot = tf.feature_column.indicator_column(Cowork_Abroad)
feature_columns.append(Cowork_Abroad_one_hot)

#Cowork_etc
df["Cowork_etc"] = df["Cowork_etc"].apply(str)
Cowork_etc = tf.feature_column.categorical_column_with_vocabulary_list(
      'Cowork_etc', df['Cowork_etc'].unique() )
Cowork_etc_one_hot = tf.feature_column.indicator_column(Cowork_etc)
feature_columns.append(Cowork_etc_one_hot)




In [12]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [13]:
feature_columns

[NumericColumn(key='Log_RnD_Fund', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='Log_Duration', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_of_SCI', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_of_Paper', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_Patent_App', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_Patent_Reg', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_of_Korean_Patent', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_of_Inter_Patent', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_of_Patent', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='STP_Code_1_Weight', sh

# train, test, val split

In [14]:
df.columns

Index(['index', 'Project_ID', 'Year', 'N_of_SCI', 'N_of_Paper', 'N_Patent_App',
       'N_Patent_Reg', 'N_of_Korean_Patent', 'N_of_Inter_Patent',
       'N_of_Patent', 'Multi_Year', 'RnD_Org', 'Region', 'STP_Code_11',
       'STP_Code_1_Weight', 'STP_Code_21', 'STP_Code_2_Weight',
       'Application_Area_1', 'Application_Area_1_Weight', 'Application_Area_2',
       'Application_Area_2_Weight', 'Application_Area_3',
       'Application_Area_3_Weight', 'Green_Tech', 'SixT_2', 'Econ_Social',
       'National_Strategy_2', 'RnD_Stage', 'Cowork_Cor', 'Cowork_Uni',
       'Cowork_Inst', 'Cowork_Abroad', 'Log_RnD_Fund', 'Log_Duration',
       'Cowork_etc', 'Sales', 'Income', 'Asset', 'Capital',
       'Sales_Income_Ratio', 'Asset_Income_Ratio', 'Sales_Operation_Ratio',
       'Expense_Ratio', 'Debt_Ratio', 'IPO', 'Comp_Type', 'Listed_Market',
       'Administration', 'External_Audit', 'Survival', 'Venture', 'Innobiz',
       'Mainbiz', 'Employees', 'Closed', 'Ten_Industry_1', 'Ten_Industry_11

In [15]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

In [16]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# train
x_train = train_df.loc[:,['Year', 'N_of_SCI', 'N_of_Paper', 'N_Patent_App', 'N_Patent_Reg',
       'N_of_Korean_Patent', 'N_of_Inter_Patent', 'N_of_Patent', 'Multi_Year',
       'RnD_Org', 'Region', 'STP_Code_11', 'STP_Code_1_Weight', 'STP_Code_21',
       'STP_Code_2_Weight', 'Application_Area_1', 'Application_Area_1_Weight',
       'Application_Area_2', 'Application_Area_2_Weight', 'Application_Area_3',
       'Application_Area_3_Weight', 'Green_Tech', 'SixT_2', 'Econ_Social',
       'National_Strategy_2', 'RnD_Stage', 'Cowork_Cor', 'Cowork_Uni',
       'Cowork_Inst', 'Cowork_Abroad', 'Log_RnD_Fund', 'Log_Duration',
       'Cowork_etc', 'Sales', 'Income', 'Asset', 'Capital',
       'Sales_Income_Ratio', 'Asset_Income_Ratio', 'Sales_Operation_Ratio',
       'Expense_Ratio', 'Debt_Ratio', 'IPO', 'Comp_Type', 'Listed_Market',
       'Administration', 'External_Audit', 'Survival', 'Venture', 'Innobiz',
       'Mainbiz', 'Employees', 'Closed', 'Ten_Industry_1', 'Ten_Industry_11',
       'Ten_Industry_111', 'Researcher']]
y_train = train_df.loc[:,['Comm_Success',
       'Comm_Success_Code1_4', 'Comm_Success_Code2_5', 'Comm_Success_Code3_6']]
x_train = dict(x_train)

# val
x_val = val_df.loc[:,['Year', 'N_of_SCI', 'N_of_Paper', 'N_Patent_App', 'N_Patent_Reg',
       'N_of_Korean_Patent', 'N_of_Inter_Patent', 'N_of_Patent', 'Multi_Year',
       'RnD_Org', 'Region', 'STP_Code_11', 'STP_Code_1_Weight', 'STP_Code_21',
       'STP_Code_2_Weight', 'Application_Area_1', 'Application_Area_1_Weight',
       'Application_Area_2', 'Application_Area_2_Weight', 'Application_Area_3',
       'Application_Area_3_Weight', 'Green_Tech', 'SixT_2', 'Econ_Social',
       'National_Strategy_2', 'RnD_Stage', 'Cowork_Cor', 'Cowork_Uni',
       'Cowork_Inst', 'Cowork_Abroad', 'Log_RnD_Fund', 'Log_Duration',
       'Cowork_etc', 'Sales', 'Income', 'Asset', 'Capital',
       'Sales_Income_Ratio', 'Asset_Income_Ratio', 'Sales_Operation_Ratio',
       'Expense_Ratio', 'Debt_Ratio', 'IPO', 'Comp_Type', 'Listed_Market',
       'Administration', 'External_Audit', 'Survival', 'Venture', 'Innobiz',
       'Mainbiz', 'Employees', 'Closed', 'Ten_Industry_1', 'Ten_Industry_11',
       'Ten_Industry_111', 'Researcher']]
y_val = val_df.loc[:,['Comm_Success',
       'Comm_Success_Code1_4', 'Comm_Success_Code2_5', 'Comm_Success_Code3_6']]
x_val = dict(x_val)

# test
x_test = test_df.loc[:,['Year', 'N_of_SCI', 'N_of_Paper', 'N_Patent_App', 'N_Patent_Reg',
       'N_of_Korean_Patent', 'N_of_Inter_Patent', 'N_of_Patent', 'Multi_Year',
       'RnD_Org', 'Region', 'STP_Code_11', 'STP_Code_1_Weight', 'STP_Code_21',
       'STP_Code_2_Weight', 'Application_Area_1', 'Application_Area_1_Weight',
       'Application_Area_2', 'Application_Area_2_Weight', 'Application_Area_3',
       'Application_Area_3_Weight', 'Green_Tech', 'SixT_2', 'Econ_Social',
       'National_Strategy_2', 'RnD_Stage', 'Cowork_Cor', 'Cowork_Uni',
       'Cowork_Inst', 'Cowork_Abroad', 'Log_RnD_Fund', 'Log_Duration',
       'Cowork_etc', 'Sales', 'Income', 'Asset', 'Capital',
       'Sales_Income_Ratio', 'Asset_Income_Ratio', 'Sales_Operation_Ratio',
       'Expense_Ratio', 'Debt_Ratio', 'IPO', 'Comp_Type', 'Listed_Market',
       'Administration', 'External_Audit', 'Survival', 'Venture', 'Innobiz',
       'Mainbiz', 'Employees', 'Closed', 'Ten_Industry_1', 'Ten_Industry_11',
       'Ten_Industry_111', 'Researcher']]
y_test = test_df.loc[:,['Comm_Success',
       'Comm_Success_Code1_4', 'Comm_Success_Code2_5', 'Comm_Success_Code3_6']]
x_test = dict(x_test)

#모델생성

In [17]:
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import ModelCheckpoint

In [18]:
METRICS = [
      keras.metrics.AUC(name='auc', multi_label=True, num_labels=4),

      tfa.metrics.F1Score(
      num_classes = 4,
      average = 'micro',
      name ='f1_score',
      threshold = 0.5)

]

In [19]:
cb = tf.keras.callbacks.EarlyStopping( monitor='val_loss', min_delta = 0,  patience=500, verbose=1, mode='min', baseline=None )


def creat_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns=feature_columns),
    tf.keras.layers.Dense(units=300, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(units=300, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(units=300, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(units=4, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss= tfa.losses.SigmoidFocalCrossEntropy(gamma = 2, alpha=[0.5, 0.965, 0.6, 0.93]),
                metrics=METRICS)
  
  return model


model = creat_model()


model.fit(
    x_train, y_train,
    batch_size=256,
    epochs=2000,
    validation_data=(x_val, y_val),
    callbacks=cb
)

y_pred = (model.predict(x_test) > 0.5).astype("int32")

y_true = test_df.loc[:,['Comm_Success','Comm_Success_Code1_4', 'Comm_Success_Code2_5',
       'Comm_Success_Code3_6']]  
y_true = np.array(y_true)
print('')
print('')
print('')
print('-----------------------테스트 결과---------------------------')
F1Score= model.evaluate(x_test, y_test)
print('   precision            recall               f1')
print(sklearn.metrics.precision_recall_fscore_support(y_true, y_pred, average='micro'))
print('')
print('')
print('')
# 모델 저장 경로
model.save('/gdrive/MyDrive/Colab Notebooks/DNN/최종본/모델저장/기업_최종')

Epoch 1/2000
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
123/123 [==============================] - ETA: 0s - loss: 0.4350 - auc: 0.5316 - f1_score: 0.3682WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'Year': <tf.Tensor 'ExpandDims_56:0' shape=(None, 1) dtype=string>, 'N_of_SCI': <tf.Tensor 'ExpandDims_37:0' shape=(None, 1) dtype=float64>, 'N_of_Paper': <tf.Tensor 'ExpandDims_35:0' shape=(None, 1) dtype=float64>, 'N_Patent_App': <tf.Tensor 'ExpandDims_31:0' shape=(None, 1) dtype=float64>, 'N_Patent_Reg': <tf.Tensor 'ExpandDims_32:0' shape=(None, 1) dtype=float64>, 'N_of_Korean_Patent': <tf.Tensor 'ExpandDims_34:0' shape=(None, 1) dtype=float64>, 'N_of_Inter_Patent': <tf.Tensor 'ExpandDims_33:0' shape=(None, 1) dtype=float64>, 'N_of_Patent': <tf.Tensor 'ExpandDims_36:0' shape=(None, 1) dtype=float64>, 'Multi_Year': <tf.Tensor 'ExpandDims_30:

Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


INFO:tensorflow:Assets written to: /gdrive/MyDrive/Colab Notebooks/DNN/최종본/모델저장/기업_최종/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/Colab Notebooks/DNN/최종본/모델저장/기업_최종/assets


In [20]:
y_pred = pd.DataFrame(y_pred)
y_true = pd.DataFrame(y_true)

y_true.columns = ['Comm_Success', 'Comm_Success_Code1_4', 'Comm_Success_Code2_5','Comm_Success_Code3_6']
y_pred.columns = ['predict', 'predict1_4', 'predict2_5','predict3_6']


df_result = pd.concat([y_true, y_pred], axis=1)

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score



labels1 = df_result['Comm_Success']	# 실제 labels
guesses1 = df_result['predict']	# 에측된 결과


labels4 = df_result['Comm_Success_Code1_4']	# 실제 labels
guesses4 = df_result['predict1_4']	# 에측된 결과

labels5 = df_result['Comm_Success_Code2_5']	# 실제 labels
guesses5 = df_result['predict2_5']	# 에측된 결과

labels6 = df_result['Comm_Success_Code3_6']	# 실제 labels
guesses6 = df_result['predict3_6']	# 에측된 결과

print('------------------------Comm_Success-----------------')
print('정확도', accuracy_score(labels1, guesses1))	
print('recall',recall_score(labels1, guesses1))	
print('precision',precision_score(labels1, guesses1))
print('f1', f1_score(labels1, guesses1))
print('예측_실제 순')
print('True_True', len(df_result[(df_result.predict == 1) & (df_result.Comm_Success == 1)]))
print('True_Falses', len(df_result[(df_result.predict == 1) & (df_result.Comm_Success == 0)]))
print('False_True', len(df_result[(df_result.predict == 0) & (df_result.Comm_Success == 1)]))
print('False_False', len(df_result[(df_result.predict == 0) & (df_result.Comm_Success == 0)]))

print('')
print('')
print('')


print('------------------------Comm_Success_Code1_4-----------------')
print('정확도', accuracy_score(labels4, guesses4))	
print('recall',recall_score(labels4, guesses4))	
print('precision',precision_score(labels4, guesses4))
print('f1', f1_score(labels4, guesses4))
print('예측_실제 순')
print('True_True', len(df_result[(df_result.predict1_4 == 1) & (df_result.Comm_Success_Code1_4 == 1)]))
print('True_Falses', len(df_result[(df_result.predict1_4 == 1) & (df_result.Comm_Success_Code1_4 == 0)]))
print('False_True', len(df_result[(df_result.predict1_4 == 0) & (df_result.Comm_Success_Code1_4 == 1)]))
print('False_False', len(df_result[(df_result.predict1_4 == 0) & (df_result.Comm_Success_Code1_4 == 0)]))

print('')
print('')
print('')

print('---------------Comm_Success_Code2_5--------------------')
print('정확도', accuracy_score(labels5, guesses5))	
print('recall',recall_score(labels5, guesses5))	
print('precision',precision_score(labels5, guesses5))
print('f1', f1_score(labels5, guesses5))
print('예측_실제 순')
print('True_True', len(df_result[(df_result.predict2_5 == 1) & (df_result.Comm_Success_Code2_5 == 1)]))
print('True_Falses', len(df_result[(df_result.predict2_5 == 1) & (df_result.Comm_Success_Code2_5 == 0)]))
print('False_True', len(df_result[(df_result.predict2_5 == 0) & (df_result.Comm_Success_Code2_5 == 1)]))
print('False_False', len(df_result[(df_result.predict2_5 == 0) & (df_result.Comm_Success_Code2_5 == 0)]))


print('')
print('')
print('')

print('-----------------------Comm_Success_Code3_6-------------------------')
print('정확도', accuracy_score(labels6, guesses6))	
print('recall',recall_score(labels6, guesses6))	
print('precision',precision_score(labels6, guesses6))
print('f1', f1_score(labels6, guesses6))
print('예측_실제 순')
print('True_True', len(df_result[(df_result.predict3_6 == 1) & (df_result.Comm_Success_Code3_6 == 1)]))
print('True_Falses', len(df_result[(df_result.predict3_6 == 1) & (df_result.Comm_Success_Code3_6 == 0)]))
print('False_True', len(df_result[(df_result.predict3_6 == 0) & (df_result.Comm_Success_Code3_6 == 1)]))
print('False_False', len(df_result[(df_result.predict3_6 == 0) & (df_result.Comm_Success_Code3_6 == 0)]))

------------------------Comm_Success-----------------
정확도 0.8399714780482835
recall 0.8467904723455794
precision 0.8378270421410026
f1 0.8422849111534986
예측_실제 순
True_True 4195
True_Falses 812
False_True 759
False_False 4051



------------------------Comm_Success_Code1_4-----------------
정확도 0.8658449628195987
recall 0.40483383685800606
precision 0.10685805422647528
f1 0.16908517350157728
예측_실제 순
True_True 134
True_Falses 1120
False_True 197
False_False 8366



---------------Comm_Success_Code2_5--------------------
정확도 0.7553223999185087
recall 0.7342842049656629
precision 0.6657088122605364
f1 0.6983170057774427
예측_실제 순
True_True 2780
True_Falses 1396
False_True 1006
False_False 4635



-----------------------Comm_Success_Code3_6-------------------------
정확도 0.82998879494754
recall 0.5615292712066906
precision 0.2652370203160271
f1 0.36029129934840937
예측_실제 순
True_True 470
True_Falses 1302
False_True 367
False_False 7678
